## Optimization

The optimization problem is best described as the search of a local maximum or minimum value of a scalar-valued function $f(x)$.  This search may be performed for all possible input values in the domain of $f$ (and in this case we refer to this problem as an *unconstrained optimization*), or for a specific sub-domain (and we refer to this other problem as a *constrained optimization*).  In this section we are going to explore both modalities in several settings.

### Unconstrained Optimization for univariate functions

We focus on the search for local minima of a function $f(x)$ in an interval $[a, b]$ (the search for local maxima can then be regarded as the search of the local minima of the function $-f(x)$ in the same interval).  For this task, we have the routine `minimize_scalar` in the module `scipy.optimize`.  It accepts as obligatory input a univariate function $f(x)$, together with a search method.

Most search methods are based on the idea of _bracketing_ that we used for root finding, although the concept of bracket is a bit different in this setting: In this case, a good _bracket_ is a triple $x < y < z$ where $f(y)$ is less than both $f(x)$ and $f(z)$.  If the function is continuous, its graph presents a U-shape on a bracket.  This guarantees the existence of a minimum inside of the sub-interval $[x, z]$.  A successful bracketing method will look, on each successive step, for the target extremum in either $[x, y]$, or $[y, z]$.

Let us construct a fast bracketing method for testing purposes:  Assume we have as initial bracket $a < c < b$.  By quadratic interpolation, we construct a parabola through the points $\big(a, f(a)\big)$, $\big(c, f(c)\big)$ and $\big(b, f(b)\big)$.  Because of the U-shape condition, there must be a minimum (easily computable) for the interpolating parabola, say $\big(d, f(d)\big)$.  It is not hard to prove that the value $d$ lies between the midpoints of the subintervals $[a, c]$, and $[c, b]$.  We will use this point $d$ for our next bracketing step.  For example, if it happens that $c < d$, then the next bracket will be either $c < d < b$, or $a < c < d$.   Easy enough. let us implement this method:

In [2]:
In [1]: import numpy as np; \
   ...: from scipy.interpolate import lagrange; \
   ...: from scipy.optimize import OptimizeResult, minimize_scalar

In [2]: def good_bracket(func, bracket):
   ...:     a, c, b = bracket
   ...:     return (func(a) > func(c)) and (func(b) > func(c))
   ...:

In [3]: def parabolic_step(f, bracket):
   ...:     stop = False
   ...:     funcalls = 0
   ...:     niter = 0
   ...:     while not stop:
   ...:         niter += 1
   ...:         interpolator = lagrange(np.array(bracket), f(np.array(bracket)))
   ...:         funcalls += 3
   ...:         a, b, c = interpolator.coeffs
   ...:         d = -0.5*b/a
   ...:         if np.allclose(bracket[1], d):
   ...:             minima = d
   ...:             stop = True
   ...:         elif bracket[1] < d:
   ...:             newbracket = [bracket[1], d, bracket[2]]
   ...:             if good_bracket(func, newbracket):
   ...:                 bracket = newbracket
   ...:             else:
   ...:                 bracket = [bracket[0], bracket[1], d]
   ...:         else:
   ...:             newbracket = [d, bracket[1], bracket[2]]
   ...:             if good_bracket(func, newbracket):
   ...:                 bracket = newbracket
   ...:             else:
   ...:                 bracket = [bracket[0], d, bracket[1]]
   ...:     return OptimizeResult(fun=func(x), x=minima, nit=niter, nfev=funcalls)
   ...:

> The output of any minimizing method must be an `OptimizeResult` object, with at least the attribute `x` (the solution to the optimization problem).  In the example we have just run, the attributes coded in this method are `x`, `fun` (the evaluation of `f` at that solution), `nit` (number of iterations), and `nfev` (number of functions evaluations needed).

Let us run this method over a few examples:

In [10]:
In [4]: def g(x): return -np.exp(-x)*np.sin(x); \
   ...: f = np.vectorize(lambda x: max(1-x, 2+x))

In [5]: print good_bracket(f, [-1, -0.5, 1])

In [6]: print minimize_scalar(f, bracket=[-1, -0.5, 1], method='parabolic_step')

In [7]: print good_bracket(g, [0, 1.2, 1.5])

In [8]: print minimize_scalar(g, bracket=[0,1.2,1.5], method=parabolic_step)

True


ValueError: Unknown solver parabolic_step

In [6]:
minimize_scalar?
